In [ ]:
# 20170415 attempt: case.518.com -> 500 Internal Server Error

In [179]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import config

account = config.account
pwd = config.pd

In [208]:
def get_518_login_session(br):
    
    # fill info to get login session
    br.get('http://case.518.com.tw/member-logout.html')
    br.get('https://case.518.com.tw/member-login.html')
    br.find_element_by_id('account').send_keys(account)
    br.find_element_by_id('pwd').send_keys(pwd)
    br.find_element_by_xpath('//*[@id="content-inner"]/form/div/p[4]/input').click()

    br.get('http://case.518.com.tw/casepage-index.html')
    
    s = requests.Session()
    for c in br.get_cookies():
        s.cookies.set(c['name'], c['value'])
    return s

br = webdriver.Chrome('./drivers/chromedriver_mac64')
s = get_518_login_session(br)
br.quit()

In [187]:
def get_case_list_df_by_page_url(page_url):
    res = s.get(page_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # parse needed data
    titles = [a.text.strip() for a in soup.select('li.title div a')]
    links = [a.get('href') for a in soup.select('li.title div a')]
    propose_amount = [int(a.text.strip()) for a in soup.select('li.active a')]
    budgets = [s.text.strip() for s in soup.select('li.budget span')]
    views = [int(s.text.strip()) for s in soup.select('li.view span')]
    locations = [s.text.strip() for s in soup.select('li.status span')]
    post_dates = [s.text.strip() for s in soup.select('time')]

    df = pd.DataFrame({
        'title': titles,
        'link': links,
        'propose_amount': propose_amount,
        'budget': budgets,
        'view': views,
        'location': locations,
        'post_date': post_dates
    })
    time.sleep(1)
    return df

In [209]:
def get_final_page(url):
    soup = BeautifulSoup(s.get(url).text, 'html.parser')
    return int(soup.select_one('span.pagecountnum span').text\
                                                        .split('/')[1]\
                                                        .strip())

web_section_url = 'http://case.518.com.tw/casepage-index.html?srh=KWCC20MMTA7MM'
final_page = get_final_page(web_section_url)

frame = []
pages = range(1, final_page+1)
for page in tqdm(pages):
    page_url = web_section_url.replace('.html?', 
                                       '.html?page={}&'.format(page))
    _df = get_case_list_df_by_page_url(page_url)
    frame.append(_df)
    
basic_info_df = pd.concat(frame).reset_index(drop=True)

100%|██████████| 31/31 [00:48<00:00,  1.60s/it]


In [211]:
def get_case_info_by_link(link):
    """
    return tuple contains
    link, content, location, office, time_limit and owner_positive
    """
    res = s.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')

    content = soup.select_one('dd.conttext p').text.strip()
    try:
        office = soup.select('dd.loc')[1].text.strip()
    except Exception as e:
        office = np.nan
        
    time_limit = soup.select_one('dd.date').text.strip()
    owner_positive = soup.select_one('p.positive').text.strip()

    # time.sleep(1)
    return (link, content, office, time_limit, owner_positive)

rows = []
for link in tqdm(basic_info_df['link']):
    row = get_case_info_by_link(link)
    rows.append(row)

advance_info_columns = ['link', 'content', 'office', 
                        'time_limit', 'owner_positive']
advance_info_df = pd.DataFrame(rows, columns=advance_info_columns)
rs_df = basic_info_df.merge(advance_info_df, on='link')


100%|██████████| 618/618 [13:12<00:00,  1.16s/it]


In [227]:
rs_df.to_csv('../data/case518_web.tsv', sep='\t',
                                          index=False, 
                                          encoding='utf-8')

rs_df.to_json('../data/case518_web.json', force_ascii=False, 
                                            orient='records')